In [3]:
!git clone https://github.com/facebookresearch/detectron2.git

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [4]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

2.2.2+cpu False


'gcc' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
cd detectron2

c:\Users\DELL\OneDrive\Desktop\MachineLearning\face-extraction\detectron2


In [6]:
!pip install -e .

Obtaining file:///C:/Users/DELL/OneDrive/Desktop/MachineLearning/face-extraction/detectron2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for detectron2


    error: subprocess-exited-with-error
    
    × python setup.py develop did not run successfully.
    │ exit code: 1
    ╰─> [42 lines of output]
        running develop
        C:\Users\DELL\anaconda3\envs\facetry\lib\site-packages\setuptools\command\develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
        !!
        
                ********************************************************************************
                Please avoid running ``setup.py`` and ``easy_install``.
                Instead, use pypa/build, pypa/installer or other
                standards-based tools.
        
                See https://github.com/pypa/setuptools/issues/917 for details.
                ********************************************************************************
        
        !!
          easy_install.initialize_options(self)
        C:\Users\DELL\anaconda3\envs\facetry\lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecation

In [7]:
import detectron2

In [8]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [3]:
# import cv2
# import numpy as np
# import torch
# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.config import get_cfg
# from detectron2.data import MetadataCatalog

# # Load the generated image
# image_path = 'image001.png'
# image = cv2.imread(image_path)

# # Configure the model
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# cfg.MODEL.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available

# # Create predictor
# predictor = DefaultPredictor(cfg)

# # Make prediction
# outputs = predictor(image)

# # Extract instances and masks
# instances = outputs["instances"]
# masks = instances.pred_masks.to("cpu").numpy()

# # Extract face masks (assuming COCO class ID for 'person' is 0)
# face_masks = [masks[i] for i in range(len(masks)) if instances.pred_classes[i] == 0]

# # Apply the masks to the image and save the result
# for i, mask in enumerate(face_masks):
#     # Create an output image with a transparent background
#     output_image = np.zeros_like(image, dtype=np.uint8)
#     output_image[mask] = image[mask]

#     # Save the masked face region
#     cv2.imwrite(f'face_extraction.png', output_image)

# print("Faces extracted and saved successfully.")

In [9]:
import cv2

image_path = 'image001.png'
image = cv2.imread('image001.png')

In [10]:
import torch

# Configure the model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available

# Create predictor
predictor = DefaultPredictor(cfg)

[07/13 15:27:21 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


In [11]:
print(image)

None


In [1]:
outputs = predictor(image)
instances = outputs["instances"]
masks = instances.pred_masks.to("cpu").numpy()
classes = instances.pred_classes.to("cpu").numpy()

NameError: name 'predictor' is not defined

In [9]:
instances

Instances(num_instances=2, image_height=638, image_width=524, fields=[pred_boxes: Boxes(tensor([[ 60.6096,  26.1940, 442.0771, 635.2701],
        [198.9969,   2.9485, 309.9710,  56.7440]], device='cuda:0')), scores: tensor([0.9986, 0.6880], device='cuda:0'), pred_classes: tensor([0, 0], device='cuda:0'), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
   

In [10]:
import numpy as np
from detectron2.data import MetadataCatalog

# Verify the class ID for 'person'
person_class_id = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes.index('person')

# Extract masks for 'person'
face_masks = [masks[i] for i in range(len(masks)) if classes[i] == person_class_id]

# Combine the masks if multiple instances are found
combined_mask = np.sum(face_masks, axis=0)

In [11]:
output_image = np.zeros_like(image)
output_image[combined_mask > 0] = image[combined_mask > 0]

# Optionally, add transparency where the mask is not present
output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2BGRA)
output_image[combined_mask == 0, 3] = 0  # Set alpha channel to 0 for non-mask regions

In [12]:
output_path = 'extracted_face.png'
cv2.imwrite(output_path, output_image)

# Display the result
# cv2.imshow('Extracted Face', output_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [12]:
!pip install opencv-python mediapipe numpy

  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/50.8 MB 3.6 MB/s eta 0:00:14
   ---------------------------------------- 0.5/50.8 MB 5.6 MB/s eta 0:00:09
    --------------------------------------- 1.0/50.8 MB 7.5 MB/s eta 0:00:07
   - -------------------------------------- 1.6/50.8 MB 9.5 MB/s eta 0:00:06
   - -------------------------------------- 2.3/50.8 MB 10.6 MB/s eta 0:00:05
   -- ------------------------------------- 3.0/50.8 MB 11.3 MB/s eta 0:00:05
   -- ------------------------------------- 3.6/50.8 MB 11.9 MB/s eta 0:00:04
   --- ------------------------------------ 4.2/50.8 MB 12.2 MB/s eta 0:00:04
   --- ------------------------------------ 4.9/50.8 MB 12.6 MB/s eta 0:00:04
   ---- ----------------------------------- 5.6/50.8 MB 12.6 MB/s eta 0:00:04
   ---- ----------------------------------- 6.3/50.8 MB 12.5 MB/s eta 0:00:04
   ----

In [13]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Selfie Segmentation
mp_selfie_segmentation = mp.solutions.selfie_segmentation
selfie_segmentation = mp_selfie_segmentation.SelfieSegmentation(model_selection=1)

# Initialize OpenCV face detector (Haar cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect face and remove background
def detect_face_remove_background(frame):
    # Convert the image from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get the mask with the segmentation
    result = selfie_segmentation.process(rgb_frame)
    mask = result.segmentation_mask

    # Convert mask to binary format
    condition = mask > 0.5
    
    # Prepare the background
    background = np.zeros(frame.shape, dtype=np.uint8)
    background[:] = (0, 0, 0)  # Black background
    
    # Combine the frame and background based on the mask
    output_frame = np.where(condition[..., None], frame, background)
    
    # Detect faces in the output frame
    gray_frame = cv2.cvtColor(output_frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Draw rectangles around faces
    for (x, y, w, h) in faces:
        cv2.rectangle(output_frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    return output_frame

# Open the video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Process the frame
    output_frame = detect_face_remove_background(frame)
    
    # Display the result
    cv2.imwrite('image.png', output_frame)
    
    # Break the loop on 'q' key press
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Release the resources
cap.release()
# cv2.destroyAllWindows()